# This classification model is based on MultiNomialNB 
# Pay attention to cleanning procedure

In [1]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re
import os
import json
import csv
import string
import tqdm
import pickle

# Preparing the categorized data to classification

In [2]:
def check_if_key_exists(input_key,input_collection):
    
    """
    
    checks if key exists on the collection
    
    :param input_key - the key to search on the collection
    :param input_collection - the collection
    
    :returns True if key exists, else False
    
    """
    
    key_found = False
    for key in input_collection.keys():
        if key == input_key:
            key_found = True
            break
    
    return key_found

In [3]:
def get_sub_categories_count(input_category:str, input_path:str):
    
    """
    
    checks how many subcategories exists to an input category.
    
    :param input_category - a string describes the category
    :param input_path - the verdicts files path
    
    :returns a dictionary which has the category names as keys and the occurences count as values
    
    """
    
    #initialization
    category_dict = dict()
    
    # iterate on the files on the input path
    for scanned_file in os.scandir(input_path):
        
        # look for directories only due to the fact that all the extracted verdicts
        # are already have been set to the right directory tree
        
        if scanned_file.is_dir() == True:
            
            # split the directory name by ' - ' - that is how the justice department 
            # have been decided to seperate the category and the sub category
            
            dir_name = os.path.basename(scanned_file)
            splitted_category = str(dir_name).split(" - ")
            
            # check the len of the splitted catregory
            # there are two main representations, one sub or two sub
            if splitted_category[0] == input_category:
                if len(splitted_category) == 3:
                    for first_sub in os.scandir(scanned_file):
                       
                        if check_if_key_exists(splitted_category[1]+ " - " + splitted_category[2],category_dict):
                            category_dict[splitted_category[1]+ " - " + splitted_category[2]] += 1
                        else:
                            category_dict[splitted_category[1]+ " - " + splitted_category[2]] = 1
                else:
                    for first_sub in os.scandir(scanned_file):
                        
                        if check_if_key_exists(splitted_category[1],category_dict):
                            category_dict[splitted_category[1]] += 1
                        else:
                            category_dict[splitted_category[1]] = 1
            
    category_dict[input_category + " Total"] = sum(category_dict.values())
    
    return category_dict


In [4]:
def get_categories_count(input_path:str):
    
    """
    
    counts the quantity of main categories
    
    :param input_path - a string holds the path to the root folder
    
    :returns a dictionary with the names of the 
     categories and the verdicts present for each and every of them
    
    """
    
    # initalization
    category_dict = dict()
    
    # iterate on the folder
    for scanned_file in os.scandir(input_path):
        
        # look for directories only
        if scanned_file.is_dir() == True:
            
            # look for the main category only, for example: 'Civil'
            dir_name = os.path.basename(scanned_file)
            splitted_category = str(dir_name).split(" - ")
            cur_category = splitted_category[0]
            
            # advance counter according to the findings
            for sub_cat in os.scandir(scanned_file):
                if check_if_key_exists(cur_category,category_dict):
                    category_dict[cur_category] += 1
                else:
                    category_dict[cur_category] = 1
    
    return category_dict


In [5]:
# dest_path = 'C:/Users/Itai Ofir/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
# test_category = 'Civil'
# result_sub_cat_dict = get_sub_categories_count(test_category,dest_path)
# result_cat_dict = get_categories_count(dest_path)
# print_dict(result_sub_cat_dict)
# print(" - - - - - - - - - - - - ")
# print_dict(result_cat_dict)

In [6]:
def print_dict(input_dict):
    
    """
    
    prints given dictionary
    
    :param input_dict - dictionary
    
    """
    
    for key in input_dict:
        print(key + " : " + str(input_dict[key]))

In [7]:
# test_category = 'Administrative'
# dest_path = 'C:/Users/Itai Ofir/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
# calculate_required_categories_count_ratio(test_category,dest_path)

# create train and test based on previous dicts

In [8]:
# def make_train_and_test_frames(input_category,input_category_count_dictionary,verdicts_path):
#     appereances_ratio_dict = input_category_count_dictionary.copy()
#     total_tagged_verdicts = int(sum(input_category_count_dictionary.values()))
#     for key,val in zip(input_category_count_dictionary.keys(),input_category_count_dictionary.values()):
#         appereances_ratio_dict[key] = val/total_tagged_verdicts
#     print_dict(appereances_ratio_dict)

In [9]:
dest_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
test_category = 'Civil'
result_sub_cat_dict = get_sub_categories_count(test_category,dest_path)
result_cat_dict = get_categories_count(dest_path)
print_dict(result_sub_cat_dict)
print(" - - - - - - - - - - - - - - - - - - - - - ")
print_dict(result_cat_dict)
print(" - - - - - - - - - - - - - - - - - - - - - ")


Bankruptcy : 1028
Contracts : 1085
Corporate : 1
Equity and Trusts : 246
Fiscal : 1
Intellectual Property : 105
Land : 1605
other : 1628
Procedure and Evidence : 758
Property : 946
Torts : 644
Civil Total : 8047
 - - - - - - - - - - - - - - - - - - - - - 
Administrative : 9959
Civil : 8047
Constitutional : 1932
Criminal : 4410
Family : 148
International law : 120
Labor and Employment : 532
National security, military, and the territories : 2943
Religious : 224
Social security, Health Insurance, Pension : 310
 - - - - - - - - - - - - - - - - - - - - - 


# creating train,test df of specified category

In [10]:
def create_train_test_of_specified_verdict_category(input_category:str,input_path:str, desired_test_size):
    
    """
    
    based on the root directory tree, this method will make a data frame for 
    every category which has the exact amount of matched verdicts vs non matched.
    for instance - 1000 'Civil' verdicts vs 1000 mixture of 'Administrative'
    , 'Constitutional' and so on.
    
    :param input_category - a string that holds the category name - 
     Attention! this name must be identical to the name present on the root directory
     
    :param input_path - a string that holds the root directory path
    
    :param desired_test_size - a number between 0-1 that corresponds to the 
     precentage of test data that will be created and returned.
     needless to say that will fix the train precentage as well.
    
    :returns a data frame for the train and test data for the category and non category
     for instance, 1000 Civil verdicts will yield a 70%-30% division.
    
    :returns a data frame with only id and verdict text as well.
    
    keep in mind that a verdict can, and will, classify as two categories if it exists on both folders
    
    """
    
    # dataframes initialization
    verdicts_train_df_input_category = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    verdicts_test_df_input_category = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    verdicts_train_df_not_input_category = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    verdicts_test_df_not_input_category = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    temp_df_cat = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    temp_df_not_cat = pd.DataFrame(columns = ["Veredict_ID", "Verdict_Text", "Category"])
    df_tuple = [temp_df_cat,temp_df_not_cat]
    
    # tuple initialization
    # index 1 is the category df, 0 is the opposite
    for file in os.scandir(input_path):
        if file.is_dir():
            dir_name = str(os.path.basename(file)).split(" - ")[0]
            cur_cat = 0
            if dir_name == input_category:
                cur_cat = 1
            for verdict in os.scandir(file):
                if verdict.is_dir():
                    for sub_folder in os.scandir(verdict):
                        df_tuple[cur_cat] = df_tuple[cur_cat].append(make_temp_df(sub_folder,cur_cat),ignore_index = True)
                else:
                    df_tuple[cur_cat] = df_tuple[cur_cat].append(make_temp_df(verdict,cur_cat),ignore_index = True)

    # making train-test split from the pre initialized tuple
    verdicts_train_df_input_category,verdicts_test_df_input_category = train_test_split(df_tuple[1], test_size=desired_test_size)
    verdicts_train_df_not_input_category,verdicts_test_df_not_input_category = train_test_split(df_tuple[0], test_size=desired_test_size)
    
    # adjusting the size of each df
    if verdicts_train_df_input_category.shape[0] > verdicts_train_df_not_input_category.shape[0]:
        verdicts_train_df_input_category = verdicts_train_df_input_category.sample(verdicts_train_df_not_input_category.shape[0])
    else:
        verdicts_train_df_not_input_category = verdicts_train_df_not_input_category.sample(verdicts_train_df_input_category.shape[0])
    
    if verdicts_test_df_input_category.shape[0] > verdicts_test_df_not_input_category.shape[0]:
        verdicts_test_df_input_category = verdicts_test_df_input_category.sample(verdicts_test_df_not_input_category.shape[0])
    else:
        verdicts_test_df_not_input_category = verdicts_test_df_not_input_category.sample(verdicts_test_df_input_category.shape[0])
    
    # populating the returned df
    returned_train_df = verdicts_train_df_input_category.append(verdicts_train_df_not_input_category,ignore_index=True)
    returned_test_df = verdicts_test_df_input_category.append(verdicts_test_df_not_input_category,ignore_index=True)
    
    # returning the full dfs and the id-text only variable as well
    return returned_train_df,returned_test_df,returned_train_df.drop(["Veredict_ID", "Verdict_Text"],axis=1),returned_test_df.drop(["Veredict_ID", "Verdict_Text"],axis=1)

In [11]:
def get_verdict_summary_and_id(path:str):
    
    """
    
    extracts verdict summary and id from a specified verdict
    
    :param path - a string that holds the verdict path
    
    :returns the verdict summary-string
    
    :returns the verdict id-string
    
    """
    
    with open(path,"r",encoding="utf-8") as json_file:
        json_dict = json.load(json_file)
        verdict_summary = json_dict["_source"]["doc"]["Doc Details"]["סיכום"]
        verdict_id = json_dict["_id"]
    return verdict_summary,verdict_id

In [12]:
def make_temp_df(path:str, cur_cat:str):
    
    """
    
    makes temporary df which holds the current extracted verdict and summary
    
    :param path - a string that holds the verdict path
    
    :param cur_cat - a string the holds the current category
    
    :returns a data frame that corresponds to the main data frame structre
    
    """
    
    cur_verdict,cur_id = get_verdict_summary_and_id(path)
    
    return pd.DataFrame([[cur_id,cur_verdict,cur_cat]],columns=["Veredict_ID", "Verdict_Text", "Category"])


In [13]:
# input_category = 'International law'
# input_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
# desired_test_size = 0.3 
# a,b,c,d = create_train_test_of_specified_verdict_category(input_category,input_path, desired_test_size)

In [14]:
# print(a.shape)
# print(b.shape)

In [15]:
# print(b.sample(40))

In [16]:
# def clean_text(text,stopwords_path):
#     text_splitted_to_chars = [char for char in text if char not in string.punctuation]
#     joined = ''.join(text_splitted_to_chars)
#     #print(joined)
#     stopwords = list()
#     with open(stopwords_path,"r",encoding="utf-8") as csv_file:
#         csv_lines = csv.reader(csv_file,delimiter=',')
#         stopwords = list(csv_lines)
#     return [word for word in joined.split() if word not in stopwords]

In [17]:
def clean_text_no_stopwords(text:str):
    
    """
    
    eliminates stopwords only 
    
    :param text - a string to work on
    
    :returns a cleaned text without punctuation
    
    """
    
    text_splitted_to_chars = [char for char in text if char not in string.punctuation]
    joined = ''.join(text_splitted_to_chars)
   
    return joined.split()

In [18]:
    def apply_regex_rules_on_naming_csv(self, replace_with_this_str: str, input_name: str, csv_path: str):

        """

        Adds to every name hebrew prefixes that might exists on the data.

        This method is very expansive and not recommended when not necessary.

        :param replace_with_this_str - the string to be replaced with

        :param input_name - name string

        :param csv_path - the csv that contains the names to be subtracted from each and every matched column


        :returns the element after subtracting matched words

        """

        # initalization
        updated_name = input_name
        names_list = list()
        rgx_list = list()

        # open the csv file with the words to eliminate
        with open(csv_path, "r", encoding='utf-8') as csv_file:

            names_list = csv.reader(csv_file, delimiter=',')

            names = [row[0] for row in names_list]

            # add selcted prefixes
            for name in names:
                rgx_list.append(str.format("ו?" + name))
                rgx_list.append(str.format("ל?" + name))
                rgx_list.append(str.format("כ?ש?" + name))
                rgx_list.append(str.format("ה?" + name))
                rgx_list.append(str.format("ב?" + name))
                rgx_list.append(str.format("מ?" + name))
                rgx_list.append(str.format("ש?" + name))
                rgx_list.append(str.format("כ?" + name))
                rgx_list.append(name)

                if len(name.split()) > 1:
                    rgx_list.append(str.format(name.split()[0] + '-' + name.split()[1]))

                if len(name.split('-')) > 1:
                    rgx_list.append(str.format(name.split('-')[0] + ' ' + name.split('-')[1]))

            for rgx_match in rgx_list:
                # take only specific words - not substrings, \\b is the word border.
                pattern = re.compile(str.format("\\b{0}\\b", rgx_match))
                updated_name = re.sub(pattern, replace_with_this_str, input_name)

        after_elimination = ' '.join(updated_name.split())

        return after_elimination

In [19]:
# def clean_text(text):
#     text_splitted_to_chars = [char for char in text if char not in string.punctuation]
#     joined = ''.join(text_splitted_to_chars)
    
#     text_splitted_to_chars = [char for char in text_splitted_to_chars if char not in string.digits]
#     joined = ''.join(text_splitted_to_chars)

#     after_stopwords = apply_regex_rules_on_naming_csv('',joined,'C:/Users/Itai Ofir/HebrewCourtVerdictsAnalyzer/ML/extensive_stopwords_after_filter.csv')
        
#     return after_stopwords

In [20]:
def clean_text(text:str, stopwords_path:str, naming_path:str):
    
    """
    
    cleans given text
    
    eliminates all matched occurences found on the naming and stopwords files from the given string
    
    applies hebrew prefixes when searching
    
    :param text - the given text string
    
    :param stopwords_path - the path of the stopwords file
    
    :param naming_path - the path of the common namings file
    
    :returns the text after elimination
    
    """
    
    text_splitted_to_chars = [char for char in text if char not in string.punctuation]
    joined = ''.join(text_splitted_to_chars)
    
    text_splitted_to_chars = [char for char in text_splitted_to_chars if char not in string.digits]
    joined = ''.join(text_splitted_to_chars)
    
    after_stopwords = apply_regex_rules_on_naming_csv('',joined,stopwords_path)
    after_naming = apply_regex_rules_on_naming_csv('',after_stopwords,naming_path)
    
    return after_naming

In [21]:
#after = clean_text("איתי הלך לטיול , הוא גם עו'ד גם עוד וגם שופט למרות',- שה352וא חזק")

In [22]:
#after

In [23]:
def check_best_n_gram_for_each_category(input_path,categories,n_grams,dest_path):
    
    """
    
    checks the best ngram for every category
    
    tries all of values stored on 'n_grams' dict
    
    after the precedure dumps the results text to file on the dest_apth
    
    :param input_path - a string of the path where the ready to clasiify verdict are present and ordered.
    
    :param cateogires - a list of strings that represents the category names
    
    :param n_grams - a list of round numbers that will be applied as test input for the vectorizer
    
    :param dest_path - a string of the destination path - on that path the results will be written
    
    """
    
    for cat in categories:
        path=dest_path+'/'+cat+'.txt'
        with open(path,'w',encoding='utf-8') as file:
            for n_gram_val in n_grams:
                train,test,y_train,y_test = create_train_test_of_specified_verdict_category(cat,input_path)
                model_series = []
                vec = TfidfVectorizer(ngram_range = (n_gram_val,n_gram_val))
                transformed = vec.fit_transform(train["Verdict_Text"])
                model = MultinomialNB().fit(transformed,y_train.to_numpy(dtype=float).ravel())
                test_transformed = vec.transform(test["Verdict_Text"])
                y_test_predict = model.predict(test_transformed)
                file.write(str.format('\nn_gram val : {0} \n{1}\n',n_gram_val,classification_report(y_test.to_numpy(dtype=float).ravel(),y_test_predict)))

In [24]:
def train_and_dump_model_and_tfidf_vectorizer(category:str, input_path:str, model_destination_path:str,tfidf_destination_path:str ,desired_n_gram, test_size):
    
    """
    
    trains a specific category model and with a specific n-gram and dump it to the destination
    
    :param category - a string that holds the current category
    
    :param input_path - a string that holds the root folder of the categories-ordered data
    
    :param destination_path - a string that holds the path of the dumped model
    
    :param desired_n_gram - a number with the optimal n_gram for that category
    
    """
    
    train,test,y_train,y_test = create_train_test_of_specified_verdict_category(category,input_path,test_size)
    vec = TfidfVectorizer(ngram_range = (desired_n_gram,desired_n_gram))
    transformed = vec.fit_transform(train["Verdict_Text"])
    model = MultinomialNB().fit(transformed,y_train.to_numpy(dtype=float).ravel())
    
    with open(model_destination_path,'wb') as model_file:
        pickle.dump(model,model_file)
        
    with open(tfidf_destination_path,'wb') as tfidf_file:
        pickle.dump(vec,tfidf_file)

In [25]:
def automate_train_based_on_specific_ngram(categories, input_path:str, destination_path:str):
    
    """
    
    uses the ngram check that has been done for each category and applies the best ngram for each category
    
    then dumps the model to the destination folder
    
    :param categoires - a list of strings that holds the categories
    
    :param input_path - a string of the root readytoclassify folder
    
    :param destination_path - a string of the destination folder
    
    """
    count = 1
    try:
        
        for category in categories:
            
            print(str(count)+' / '+ str(len(categories)))
            
            if category == 'Administrative':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 9, 0.3)
            
            elif category == 'Civil':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 9, 0.3)
            
            elif category == 'Constitutional':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 4, 0.3)
            
            elif category == 'Criminal':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 6, 0.3)
    
            elif category == 'Family':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 8, 0.3)
                    
            elif category == 'International law':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 9, 0.3)       
                  
            elif category == 'Labor and Employment':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 7, 0.3)
                    
            elif category == 'National security, military, and the territories':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 2, 0.3)
                    
            elif category == 'Religious':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 8, 0.3)      
                  
            elif category == 'Social security, Health Insurance, Pension':
                train_and_dump_model_and_tfidf_vectorizer(category, input_path, destination_path+'/'+category+'.pkl',destination_path+'/'+category+'.tfidf', 7, 0.3)
            
            count += 1
    except Exception as e:
        print(e)
        

In [26]:
# input_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
# categories = ['Family','Labor and Employment','Constitutional','Administrative','Civil','Criminal','International law','National security, military, and the territories','Religious','Social security, Health Insurance, Pension']
# n_gram = [1,2,3,4,5,6,7,8,9,10]
# dest_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data'
# check_best_n_gram_for_each_category(input_path,categories,n_gram,dest_path)

In [27]:
# input_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/readytoclassify'
# categories = ['Family','Labor and Employment','Constitutional','Administrative','Civil','Criminal','International law','National security, military, and the territories','Religious','Social security, Health Insurance, Pension']
# dest_path = 'C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/models'
# automate_train_based_on_specific_ngram(categories,input_path,dest_path)

In [37]:
def load_classifiers(input_path:str):
    
    """
    
    loads the classifier from the given path and returns a dictionary with the category name as key
    and the model itself as the value
    
    :param input_path - a string the represnets the models folder
    
    :returns string-model and string-tfidf dictionary
    
    """
    
    models_dict = dict()
    tfidf_dict = dict()
    
    for file in os.scandir(input_path):
        
        if os.path.isfile(file):
            
            with open(file,'rb') as pickle_file:
                    
                    cur_model = pickle.load(pickle_file)
            
                    if os.path.basename(file).find('.pkl') != -1:
                        
                        current_category = os.path.basename(file).split('.pkl')[0]
                        models_dict[current_category] = cur_model
                    
                    if os.path.basename(file).find('.tfidf') != -1:
        
                        current_category = os.path.basename(file).split('.tfidf')[0]   
                        tfidf_dict[current_category] = cur_model
    
    return models_dict, tfidf_dict
    

In [38]:
models,tfidfs = load_classifiers('C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/data/models')

In [39]:
models['Civil']

MultinomialNB()

In [40]:
models.keys()

dict_keys(['Administrative', 'Civil', 'Constitutional', 'Criminal', 'Family', 'International law', 'Labor and Employment', 'National security, military, and the territories', 'Religious', 'Social security, Health Insurance, Pension'])

In [41]:
tfidfs['Civil']

TfidfVectorizer(ngram_range=(9, 9))

In [42]:
tfidfs.keys()

dict_keys(['Administrative', 'Civil', 'Constitutional', 'Criminal', 'Family', 'International law', 'Labor and Employment', 'National security, military, and the territories', 'Religious', 'Social security, Health Insurance, Pension'])

In [43]:
category_to_ngram = {'Administrative':9,
                          'Civil':9,
                          'Constitutional':4,
                          'Criminal':6,
                          'Family':8,
                          'International law':9,
                          'Labor and Employment':7,
                          'National security, military, and the territories':2,
                          'Religious':8,
                          'Social security, Health Insurance, Pension':7
                         }

In [44]:
def classify(verdict_summary:str, models:dict, tfidfs:dict, category_to_ngram:dict):
    
    """
    
    passing the input verdict string as the prediction input for all of the pre-trained models
    
    the highest probability will be the verdict category
    
    :param verdict_summary - a string of the verdict summary
    
    :param models - category to model dictionary
    
    :param tfidfs - category to tfidf
    
    :param - category_to_nagram - category to ngram dictionary
    
    :returns - the chosen category string
    
    """
    
    category_to_score = dict()
    score_to_category = dict()
    
    for category,model in zip(models.keys(),models.values()):
        
        cur_ngram = category_to_ngram[category]
        cur_vectorizer = tfidfs[category]
        transformed = cur_vectorizer.transform([verdict_summary])
        cur_prob = model.predict_proba(transformed)
        value = cur_prob[0][1]
        category_to_score[category] = value
        score_to_category[value] = category
    
    max_prob = max(category_to_score.values())
    
    return score_to_category[max_prob]

In [52]:
text = "בנסיבות העניין, אין מקום להורות על השבת האגרה לפי תקנות בתי-המשפט (אגרות), התשס\"ז-2007. בהליך שבכותרת התקיים דיון לפני מותב של שלושה שופטים. אף שטענות הצדדים לגוף המחלוקת לא נשמעו באותו דיון, נדרש הרכב השופטים ללמוד את התיק עובר לדיון, והתיק זכה להקצאת זמן שיפוטי ביומנו העמוס של בית המשפט. מפרוטוקול הדיון עולה כי בפתח הישיבה שהתקיימה במעמד הצדדים, התייחס הרכב השופטים למאפייניו של התיק לאחר שקרא את כתבי-בית-הדין שבו, והמליץ למערערת לחזור בה מערעורה. כך אמנם אירע. אמת, דחיית הערעור בהסכמת המערערת חסכה את הצורך בשמיעת טענות הצדדים לגוף המחלוקת ובמתן הכרעה שיפוטית. עם זאת, בשים לב לשלב הדיוני בו נדחה הערעור ולהיקף השירות השיפוטי עד אותו שלב, לא מצאתי עילה שבדין להורות על השבת האגרה (ראו: תקנה 6(ב) בצירוף עם תקנה 15 לתקנות בתי המשפט (אגרות), התשס\"ז-2007; כן ראו: ע\"א 6802/09 קמינצקי נ' הוועדה המקומית לתכנון ובניה להבים (לא פורסם, 26.1.2011)). אשר על כן, הבקשה להשבת האגרה נדחית.\nניתנה היום, י\"ד בסיוון תשע\"ב (4.6.2012).\nדנה כהן-לקח, שופטת"
text2 = "1. לא ראינו מקום להתערב בעונש שהוטל על המערער, למעט עניין אחד הנוגע למאסר על תנאי שהוטל.\n2. המערער הודה במסגרת הסדר טיעון בעבירה של חבלה חמורה בנסיבות מחמירות וכן בהחזקת סמים לצריכה עצמית. מדובר בעבירה קשה של אלימות. המערער שלף סכין ודקר את המתלונן מספר פעמים וגרם לו לחתך ארוך ועמוק, בבסיס קשת הצלעות משמאל, פצעי דקירה בחלל הצפק חתכים בבית השחי ועוד. המתלונן עבר ניתוח בהרדמה כללית.\n3. בגין שתי העבירות הוטל על המערער עונש של חמישים ושניים חודשי מאסר, מתוכם 40 חודשי מאסר בפועל והיתרה - שניים עשר חודשים - על תנאי למשך שנתיים מיום שחרורו, אם המערער יורשע בעבירת אלימות או בכל עבירה על פי פקודת הסמים המסוכנים. הופעל גם במצטבר מאסר על תנאי של שישה חודשים. בנוסף הוטל פיצוי למתלונן של 7,000 ש\"ח.\n4. בשים לב להרשעותיו הקודמות של המערער ולחומרת הפציעה העונש אינו חמור כלל ועיקר ואנו דוחים את הערעור. עם זאת, בהסכמת המדינה אנו קובעים כי התנאי לא יחול על עבירה של החזקת סם לצריכה עצמית.\nניתן היום, כ\"ו תשרי, תשע\"א (4.10.2010)."
text3 = "השופט נ' הנדל:\n1. מונח בפנינו ערעור על פסק דינו של בית המשפט המחוזי בתל אביב יפו (כב' סגן הנשיא, ד\"ר עודד מודריק) בת\"פ 4939-03-13, בו הורשע המערער בעבירה של החזקת סם שלא לצריכה עצמית, לפי סעיפים 7(א) ו 7(ג) לפקודת הסמים המסוכנים [נוסח חדש], התשל\"ג-1973. המערער נדון למאסר בפועל של 42 חודשים, ולכך נוספה הפעלת מאסר על תנאי של 10 חודשים שנגזרו עליו בתיק קודם. סה\"כ נגזרו על המערער מאסר בפועל של 52 חודשים, שנה וחצי נוספת של מאסר על תנאי, וקנס בסך 5,000 ₪ או חודשיים מאסר תמורתו.\nבפסק הדין בערכאה קמא נקבע כי למרות שמערך הראיות הקיים הינו נסיבתי, הן אמינות ומהימנות. עדויות שוטרי הסיור שכנעו את בית המשפט כי הם ראו את המערער זורק דבר נוצץ לצד, וכי דבר זה נמצא ממש לאחר מכן, במקום שצפוי היה להימצא. בית המשפט דחה תרחישים אחרים שהוצעו על ידי הסניגור, כגון הטענה כי הסמים היו שם עוד לפני כן, בשל אי הסבירות לכך שחבילת סמים בשווי של כ 20,000 ₪ תונח זרוקה בסמטה נטושה. חיזוק לגרסת התביעה נמצא בתגובת המערער לעיכובו על ידי השוטרים, כפי שיובהר בהמשך. נקבע כי מארג הראיות הכולל מציג תמונה שממנה עולה אשמתו של המערער מעבר לספק סביר בעבירת החזקת הסם שלא לצריכה עצמית.\n3. עולה מהחומר כי המערער נצפה על ידי שוטרי הסיור כשהוא משליך חפץ כסוף מידו, בשעת לילה מאוחרת, בסמוך לסמטה חשוכה. מיד לאחר מכן, בחיפוש קצר במרחק של כמה מטרים, נמצאת חבילת סם עטופה בנייר כסף בסמטה קרובה. לא רק זאת, אלא שכשנשאל המערער מדוע השליך את חבילת הסמים, משך בכתפיו והביט לשמיים, ולא הכחיש את המעשה. בחקירתו, ולאחר מכן בבית המשפט, לא הצליח המערער להניח גרסה עובדתית אחרת שתניח את הדעת. הצטברות כל הראיות לעיל מבססת את אשמתו של המערער מעבר לספק סביר, כפי שקבעה הערכאה קמא.\nבצורה ממוקדת, מעלה הסניגור שתי טענות: האחת היא כי עדויותיהם של שוטרי הסיור מלאות סתירות, וכי דו\"חות הפעולה תואמו ביניהם. ביסוד טענה זו ניסיון לערער את אמינות העדויות של השוטרים. האחרת הינה כי הנמקתו של בית משפט קמא קצרה ביותר. טענות אלו הינן בעלות משקל, ודורשות התייחסות ובחינה. ברם, אין בכוחן לעורר ספק סביר במסגרת תיק זה. אסביר.\nבאשר לדו\"חות השוטרים – עולה כי לכל הפחות אנשי המשטרה שוחחו ביניהם על האירוע טרם כתיבת תיאור המקרה על ידי כל שוטר בנפרד. חמור מכך, אחד מהדו\"חות מתייחס לדו\"חות פעולה של השוטרים האחרים שהיו במקום האירוע. מצב זה מעורר קושי, ואף קושי של ממש. מצופה כי כל שוטר יכתוב בעצמו את דו\"ח הפעולה. כוחו של הדו\"ח שהוא מהווה מעין תיעוד ישיר בסמוך לאירוע. אין בו מקום לסיעור מוחות, או להשלמה משותפת של פרטים שהתרחשו באירוע. דו\"ח פעולה שנכתב על ידי בעל תפקיד בסיום אירוע מהווה מקור מידע חשוב המשמש במקרים רבים לבירור האמת. תהליך בדיקה בין השוטרים, מעין סימפוזיון זוטא, עלול לפגוע במהימנות הנכתב, ואף מעבר לכך. הראציונל בעריכת הדו\"ח הוא הרצון לדייק, תוך הכרה בכך ששוטר עשוי להיות מעורב באירועים רבים, ולהעיד על האירוע הפרטני חודשים ארוכים לאחר המקרים. ההכרה בקושי שמלווה את תפקיד השוטר צועדת יד ביד עם הדרישה כי יקפיד שגרסתו תהיה גרסתו, ולא גרסתם. דווקא משום כך, לו הייתה ההרשעה מבוססת על הדו\"חות בלבד, היה מקום לשקול היטב את טענת הסניגור. ברם, בענייננו, הכרעת הדין מבוססת על מספר אדנים.\nראשית כאמור, המערער אינו מכחיש כי ביצע תנועת זריקה. אמנם הוא מכחיש שזרק דבר מה, אך אין הסבר לתנועה שביצע. מעבר לכך, הסם נמצא בקרבת מקום. לכך נוסיף כי מיד לאחר האירוע הגיב המערער באופן שממנו ניתן היה להבין כי הוא משלים עם העונש המגיע לו (הרים ידיו למעלה והסתכל לשמיים) ולא הכחיש את מעשהו. בנסיבות העניין יש בכך מעין ראשית הודיה. מעבר לכך, המערער אף שלח מבטים שוב ושוב לכיוון הסמטה בה נתפסו הסמים.\nסיכומו של דבר – נדמה כי הכרעת הדין מבוססת כנדרש. ההרשעה אינה נשענת לבדה על דו\"חות השוטרים, אלא כאמור על נתונים מחוץ למעגל פעילות המשטרה, כגון מקום תפיסת הסמים, ותגובת המערער לביצוע המעצר.\n5. הייתי מציע לחבריי לדחות את הערעור על שני חלקיו.\nהשופט א' שהם:\nאני מסכים.\nהשופטת ע' ברון:\nאני מסכימה.\nאשר על כן, הוחלט כאמור בפסק דינו של השופט נ' הנדל.\nניתן היום, כ\"ב בסיון התשע\"ה (9.6.2015)."
text4 = "השופט ח' מלצר:\n1. עניינה של העתירה שלפנינו בהליך בחירת מנהל לבתי הדין הרבניים. העתירה הוגשה בעיצומו של ההליך, שהסתיים בינתיים במינויו של הרב שלמה דיכובסקי (להלן: הרב דיכובסקי) כמנהל קבוע לבתי הדין הרבניים עד לשנת 2014, לאחר ששימש בתפקיד במינוי זמני משך כששה חודשים. העותרים קראו, ועודם קוראים, לביטולו של הליך הבחירה, המצוי בתחום אחריותו של שר המשפטים, הוא המשיב. העותרים מבקשים עוד כי נפרש את סעיף 13 לחוק הדיינים, התשכ\"ט-1969 (להלן: חוק הדיינים), שמכוחו ממונה מנהל בתי הדין הרבניים, כמאפשר גם לנשים להגיש מועמדותן לתפקיד. לחלופין מבקשים העותרים כי נכריז על ביטול סעיף החוק ככל שהוא פוגע בזכויות חוקתיות. כן הם עותרים שנורה למשיב לפעול באופן אקטיבי כדי לגרום לנשים להגיש מועמדות למשרה, ובנוסף כי נורה לו ליתן ייצוג הולם לנשים בוועדה המייעצת שמינה לצורך הליך הבחירה.\nלהלן נפרט את הנתונים הרלבנטיים להכרעה במכלול.\n2. בתאריך 10.6.2010 החליטה הוועדה למינוי דיינים, בראשות שר המשפטים, לקצוב את כהונתו של מנהל בתי הדין הרבניים, הרב אליהו בן-דהן (להלן: הרב בן דהן), ששימש בתפקידו למעלה משני עשורים, כך שהלה יסיים את תפקידו בתאריך 11.8.2010. בעקבות זאת החל הליך למינוי מנהל חדש לבתי הדין הרבניים, מינוי המבוצע מכוחו של סעיף 13 לחוק הדיינים, הקובע כדלקמן:\n\"(א) השר, בהסכמת נשיא בית הדין הרבני הגדול, יקבע, בתקנות או בהוראות מינהל, לפי הענין, את סדרי המינהל של בתי הדין הרבניים וימנה על ביצועם את אחד הדיינים או אדם הכשיר להיבחר רב עיר, שיהיה אחראי בפניהם; מינוי לפי סעיף קטן זה של מי שכשיר להיבחר רב עיר טעון אישור ועדת המינויים.\n(ב) מנהל בתי הדין הרבניים שאיננו דיין, מינויו אינו טעון מכרז פומבי ודינו, לענין סעיף 17 ולענין חוק גימלאות לנושאי משרה ברשויות השלטון, התשכ\"ט-1969, כדין דיין.\"\n3. אף שסעיף 13 הנ\"ל איננו מחייב לעשות כן, החליט השר להקים ועדה לאיתור מועמדים למשרת המנהל החדש (להלן: הוועדה המייעצת).\n4. בעקבות האמור לעיל פנו העותרים לשר המשפטים במכתב שבו טענו כי עליו לפתוח את תפקיד מנהל בתי הדין הרבניים בפני נשים. זאת, לגישתם, מכוח הוראות חוק יסוד: חופש העיסוק, חוק יסוד: כבוד האדם וחירותו, חוק שיווי זכויות האשה, התשי\"א-1951 (להלן: חוק שיווי זכויות האשה), וחוק שוויון הזדמנויות בעבודה, התשמ\"ח-1988 (להלן: חוק שוויון הזדמנויות בעבודה). העותרים הוסיפו וטענו כי מינוי אשה לתפקיד מנכ\"ל בתי הדין הרבניים עשוי להיות צעד ראשון לתיקון מעמדן הנחות, לפי הנטען, של הנשים בבתי הדין הרבניים ואיזון לכך שכל הדיינים בבתי הדין הרבניים הם גברים. עוד דרשו העותרים מן המשיב כי בוועדה המייעצת שתתמנה לעניין איתור המועמדים לתפקיד יהיה ייצוג הולם לנשים.\n5. בהמשך להתפתחויות הנ\"ל שר המשפטים מינה, בתאריך 22.7.2010, ועדה מייעצת, הכוללת ארבעה גברים (הרב ד\"ר רצון ערוסי, השופט (בדימ') יעקב שמעוני, הרב ד\"ר יעקב הדני, פרופ' אליאב שוחטמן) ואשה אחת (עו\"ד בת שבע שרמן-שני, המשמשת, בין היתר, כמנכ\"ל בארגון \"יד לאישה\", המסייע לנשים עגונות ומסורבות גט). למחרת היום פורסמה באתר לשכת הפרסום הממשלתית פנייה לאיתור מועמדים.\n6. בתאריך 29.7.2010 הגישו העותרים עתירה לבית משפט זה, עתירה שקדמה לזו שלפנינו (בג\"ץ 5720/10; להלן: העתירה הקודמת). בעתירה הקודמת ביקשו העותרים כי בית המשפט יורה למשיב לפרש את סעיף 13 לחוק הדיינים באופן כזה המאפשר מינויה של אשה כמנהלת של בתי הדין הרבניים, ולחלופין יקבע כי סעיף חוק זה בטל ככל שהוא פוגע בזכויות חוקתיות. כן ביקשו העותרים כי בית משפט זה יורה על הרחבת ייצוג הנשים בוועדה המייעצת, נוכח החובה לייצוג הולם על פי סעיף 6 לחוק שיווי זכויות האשה.\n7. העתירה הקודמת נדחתה על הסף בתאריך 17.8.2010 (מפי השופטים: א' גרוניס, מ' נאור ו-ע' פוגלמן). בעניין הסעד הנוגע למינויה של אִשה לתפקיד מנהלת בתי הדין הרבניים נקבע כי העתירה דנא היא מוקדמת מאחר שלפי הודעת המשיב טרם נעשתה בחינה ביחס לאלה שהגישו מועמדות לתפקיד, ולא נתקבלה כל החלטה בשאלה האם ניתן לפרש את סעיף 13 לחוק הדיינים כמאפשר מינוי אשה למשרה זו. בית המשפט הוסיף שלעותרים תהיה שמורה הזכות לפנות למשיב ואף לבית משפט זה, אם יהיה בכך צורך, לאחר שתתקבל ההחלטה בדבר המינוי.\nאשר לסעד שעניינו ייצוג הולם של נשים בוועדה המייעצת נקבע כי העותרים לא מיצו את ההליכים לשינוי הרכב הוועדה וכן כי הם לא פירטו מדוע בנסיבות העניין מינויה של אִשה אחת מבין חמישה חברים בוועדה איננו מהווה ייצוג הולם.\n8. בסמוך לפני ההכרעה בעתירה הקודמת, ומשפקעה בינתיים כהונתו של הרב בן דהן כמנהל בתי הדין הרבניים, מינה שר המשפטים את הרב דיכובסקי, דיין בקיצבה של בית הדין הרבני הגדול, למנהל בפועל של בתי הדין הרבניים. מינויו הוארך באופן זמני עד למחצית חודש פברואר 2011.\n9. לאחר מתן פסק הדין בעתירה הקודמת נמשכו הליכי בחינת המועמדים, והעותרים התמידו בפניות למשיב לקידום אפשרות מינויה של אשה לתפקיד מנהלת בתי הדין הרבניים. עמדת המשיב היתה, בין היתר, שהוועדה המייעצת הונחתה לבחון את נתוניהם של כלל המועמדים, נשים כגברים, ואם יימצא כי כישוריה של מועמדת אשה דומים לאלה של המועמדים הגברים המתאימים ביותר – יבחן המשיב, יחד עם היועץ המשפטי לממשלה, את אפשרות מינויה של אשה לתפקיד.\n12. בתאריך 1.3.2011, לאחר שלא הושגה הסכמה על מינוי מי מבין המועמדים המומלצים שהעבירה הוועדה המייעצת, מינה שר המשפטים את הרב דיכובסקי לתפקיד, בהסכמת נשיא בית הדין הרבני הגדול ובאישור הוועדה למינוי דיינים. תוקף המינוי נקבע עד לתאריך י\"ט באב התשע\"ד (15.8.2014). הודעה בדבר המינוי התפרסמה ברשומות בתאריך 12.4.2011 (י\"פ התשע\"א, עמ' 3276). שר המשפטים הודיע לרב דיכובסקי כי סופיות מינויו כפופה להכרעת בית משפט זה בעתירה.\nטענות הצדדים\n13. העותרים טוענים כי בהליך מינוי המנכ\"ל לבתי הדין הרבניים נפלו פגמים, המצדיקים את ביטולו.\nבראש הראשון של עתירתם הם טוענים כי ראוי היה לפרש את סעיף 13 לחוק הדיינים (שצוטט בפיסקה 2 לעיל) באופן שלא יגביל את מועמדותן של נשים לתפקיד ויעלה בקנה אחד עם חוקי היסוד הנ\"ל וחוק שיווי זכויות האשה. לשיטת העותרים פרשנות שכזו צריכה להוביל למסקנה כי אין כל פגם בכשירותן של נשים לתפקיד מנהלת של בתי הדין הרבניים. לחלופין טוענים העותרים כי יש להכריז על בטלותו של סעיף 13 לחוק הדיינים, ככל שהוא פוגע בזכויות חוקתיות.\nבראש השני של העתירה טוענים העותרים לפגמים בהליך בחינת המועמדים לתפקיד ובהליכי עבודתה של הוועדה המייעצת שפעלה בעניין, ובהם (לשיטת העותרים): אי מילוי החובה לפעול באופן אקטיבי לאיתור מועמדות ראויות למשרה, אי מילוי החובה לפעול לייצוג הולם של נשים בניהול בתי הדין הרבניים ואי מתן משקל מספק לעיקרון הייצוג השוויוני בהליך המינוי עצמו. עוד משיגים העותרים על כך שלא יושם עיקרון הייצוג ההולם גם בקביעת הרכב הוועדה המייעצת לשר המשפטים ונטען כי אין מנוס מלהורות למשיב למנות ועדת איתור חדשה, אשר מיוזמתה תפנה לנשים ראויות ותבחן את כל המועמדים על פי השיקולים השייכים לענין, לרבות השיקולים בדבר ייצוג הולם של נשים במגזר הציבורי בכלל ובמערכת בתי הדין הרבניים בפרט.\nראשית מדגיש המשיב כי עם מינויו של הרב דיכובסקי למנהל קבוע של בתי הדין הרבניים השתנה המצב העובדתי, מושא העתירה, באופן ההופך את העתירה דנן לעיונית. המשיב מדגיש כי לרב דיכובסקי יש כישורים, רקע וניסיון רלבנטיים מיוחדים ויוצאי דופן לתפקיד, שמקנים לו עדיפות ניכרת על פני כל מועמד פוטנציאלי אחר לתפקיד, יהיה זה גבר או אשה, ואף העותרים אינם חולקים על כך. המשיב מסביר ומציין בהקשר זה כי הרב דיכובסקי כיהן כשמונה שנים כדיין בבית דין אזורי, כחמש שנים נוספות כאב בית דין אזורי וכ-19 שנים ומחצה – כדיין בבית הדין הרבני הגדול, כל זאת בנוסף לתקופת כהונתו הזמנית כמנהל בפועל של בתי הדין הרבניים. עוד שימש הרב דיכובסקי כמרצה מן החוץ באוניברסיטאות תל אביב ובר אילן. עמדת המשיב היא שתהא פרשנותו הנכונה של סעיף 13 לחוק הדיינים אשר תהא, הרי שבשים לב לכך שהעותרים אינם יכולים להצביע על מועמדת פוטנציאלית לתפקיד מנהל בתי הדין הרבניים שהיא בעלת \"כישורים דומים\" לאלה של הרב דיכובסקי – אין כל צורך להכריע במסגרת העתירה בטענות העותרים בנוגע לסעיף 13 הנ\"ל, או באשר לתחולת סעיף 6ג לחוק שיווי זכויות האשה ביחס למשרה זו, והטענות הופכות בהקשרים שלפנינו להשגות תיאורטיות גרידא.\nעוד טוען המשיב כי דין העתירה להידחות על הסף מחמת שיהוי שנפל בהתנהלות העותרים, ומכיוון שהיה על העותרים לתקן את עתירתם לאחר מינויו של הרב דיכובסקי ולהוסיפו כמשיב לעתירה – דבר שהעותרים לא עשו.\nהמשיב מוסיף עוד כי ממילא דין העתירה להידחות גם לגוף הדברים. לגבי הטענות בדבר הרכב הוועדה המייעצת, המשיב מדגיש כי זו מונתה באופן וולנטארי, מבלי שהיתה כל חובה שבדין לעשות כן, ובנסיבות אלה אף אם תתקבל טענת העותרים וייפתח מחדש הליך המינוי – אין כל הכרח כי תמונה ועדה חדשה.\nבכל הנוגע לדרישת העותרים לפעולה אקטיבית מצד המשיב – לעודד נשים להגיש את מועמדותן למשרה, המשיב גורס כי מעולם לא ציין במודעות שפורסמו לציבור כי מדובר במשרה לגברים בלבד וכי אשה אחת אכן הגישה את מועמדותה. בהקשר זה שב ומדגיש המשיב כי אין כל מקום לפתוח עתה מחדש את הליך המינוי.\n15. העותרים ביקשו להשיב לטענות המשיב בכתב ואמנם עשו כן, וגם הרחיבו דברים בעל פה לפנינו. טענתם העיקרית היא שחרף מינויו של הרב דיכובסקי (שהעותרים אינם טוענים כי קיימת מועמדת – או מועמד – שווי-כישורים לו) – לא הפכה עתירתם לעיונית. לגישתם – אין במינויו של אדם בעל כישורים מיוחדים, ככל שיהיו, כדי לייתר את הטענות בדבר חובת המשיב לקיים הליך שוויוני הפתוח בפני נשים וחובתו לפעול למען איתור אקטיבי של מועמדות. העותרים גורסים איפוא כי \"לא בטוח\" שתיקון העתירה הכרחי על מנת לדון בטענותיהם העקרוניות, הגם שהצהירו שאם בית משפט זה יסבור כי הדבר נחוץ – הם נכונים לבקש לתקן את עתירתם לצורך חידוד השאלות שנותרו עדיין שנויות במחלוקת.\nלאחר סקירה זו של השתלשלות העניינים וטענות הצדדים הגיעה העת להכריע במכלול.\nדיון והכרעה\n16. לאחר שבחנתי את טענות הצדדים, אציע לחבריי לדחות את העתירה שלפנינו. אפרט עתה בתמציתיות את הנימוקים שבבסיס מסקנתי זאת, כאשר אפתח בטעם דיוני ואעבור לאחר מכן לטעם הענייני שבדבר.\n17. מבחינה דיונית אכן נפל פגם בהתנהלות העותרים, עת לא צירפו כמשיב לעתירה את הרב דיכובסקי לאחר שהוא נבחר לכהן כמנהל הקבוע של בתי הדין הרבניים. בזמן הגשת העתירה המצב היה אמנם שונה, שכן טרם מונה איש לתפקיד ואולם משנמסרה לעותרים עובדת המינוי הם היו צריכים לצרף את הרב דיכובסקי כמשיב, בהיותו צד נוגע בדבר, שההחלטה בעתירה עלולה לפגוע בו (ויוזכר כי המשיב טען זאת מפורשות בתגובתו המקדמית). משלא עשו העותרים כן, אף שהיו להם די והותר הזדמנויות לתקן מחדל זה, הרי שדבק פגם משמעותי בעתירתם, שיש בו, כשלעצמו, כדי להביא לדחייתה (השוו: בג\"ץ 84/82 הסתדרות פועלי אגודת ישראל נ' השר לענייני דתות, פ\"ד לז(1) 813, 817-815 (1983); בג\"ץ 828/90 סיעת \"הליכוד\" במועצת עיריית חיפה נ' מועצת עיריית חיפה, פ\"ד מה(1) 506, 516 (1991); בג\"ץ 9402/03 בוכניק נ' המועצה האזורית לתכנון ובניה (לא פורסם, 18.3.2007), פיסקה 11 לפסק הדין; בג\"ץ 2592/10 פלוני נ' פקידת סעד לסדרי דין (לא פורסם, 26.4.2010)).\n18. בנוסף לפגם הדיוני האמור, הרי שממילא אין מקום להעמיד כיום לבירור, במסגרת הנוכחית, את טענות העותרים במישור החוקתי, בכל הכרוך בפרשנותו הראויה של סעיף 13 לחוק הדיינים ואפשרות מינוייה של אשה לתפקיד בגדרו. זאת לא מחמת קיומו של מעשה עשוי (והרי המינוי הוכפף ממילא להכרעה בעתירה), כי אם משום שטענות אלה אכן הפכו עתה עיוניות, כפי שטען המשיב, שהרי בבחירה של הרב דיכובסקי למשרה אמנם מדובר במינוי של מי שאין חולק על כישוריו יוצאי הדופן והתאמתו לתפקיד. דומה שאף העותרים עצמם מסכימים לכך (ראו לדוגמה מכתבם מתאריך 11.5.2011, מש/3, כמו גם האמור בבקשתם לצו ביניים, בגדרה עתרו לכך שהרב דיכובסקי ימשיך לכהן בתפקיד – במינוי זמני). העותרים גם אינם טוענים כי האשה האחת שהגישה את מועמדותה לתפקיד (היא מנכ\"לית העותר 1) היא שוות כישורים לאלו של הרב דיכובסקי, שאז על פי טענתם היה מקום לבחון את האפשרות העקרונית למינוי אשה לתפקיד. באופן דומה – העותרים, המייצגים קשת רחבה של ארגונים הפועלים, בין היתר, לקידום מעמד הנשים ולמיגור קיפוחן ואפלייתן – אינם גורסים כי קיימת מועמדת כלשהי, אף אם לא הגישה מועמדותה לתפקיד, שכישוריה ומעלותיה דומים לאלו של הרב דיכובסקי לצורך התפקיד – מושא העתירה.\n19. למעשה כל שנותר מעתירתם של העותרים הן השאלות העיוניות שהעלו, שעיקרן: האם ניתן לפרש את סעיף 13 לחוק הדיינים כמאפשר מינויה של אשה לתפקיד מנהלת בתי הדין הרבניים, ואם לא – האם יש מקום להורות על בטלותה של הוראה זו, ככל שהיא פוגעת בזכויות חוקתיות נטענות? טענות אלה של העותרים בדבר האפלייה המובנית, לדעתם, בחוק הדיינים ודאי מעלות שאלות מורכבות ומעניינות (עיינו: Martha Minow \"Should Religious Groups Ever Be Exempt From Civil Rights Laws?\" 48 Boston College Law Review 781, 801-808 (2007); John T. Noonan, Edward M. Gaffney Religious Freedom (3rd ed., 2011), 794-796; McClure v. Salvation Army, 460 F.2d 553 (5th Cir. 1972), cert. denied, 409 US 896 (1972); Equal Employment Opportunity Commission v. Mississippi College, 626 F. 2d 477 (5th Cir. 1980); Equal Employment Opportunity Commission v. Catholic University of America, 83 F. 3d 455 (D.C. Cir. 1996) ). ברי עם זאת שככל שמדובר בהליך בחירתו הנוכחי של מנהל בתי הדין הרבניים, הרי ששאלות אלה הן עיוניות, ואינן מצדיקות את הכרעתנו לעת הזו.\n20. נוכח כל הטעמים המפורטים לעיל – אציע לחבריי כי נדחה את העתירה, בלא שנעשה בה צו להוצאות.\nהשופט (בדימ') א' א' לוי:\nאני מסכים.\nהשופט י' דנציגר:\nאני מסכים.\nהוחלט כאמור בפסק דינו של השופט ח' מלצר.\nניתן היום, א' בטבת, תשע\"ב (27.12.2011)."
res = classify(text4,models,tfidfs,category_to_ngram)
print(res)

Religious


In [55]:
with open('C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/Pipeline Python Scripts/israel_supreme_court_judges.txt','r') as file:
    with open('C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/Pipeline Python Scripts/israel_court_judges.txt','w') as new_file:
        lines = file.readlines()
        print(lines[1])
        for line in lines:
            if len(line.split()) <= 3 and len(line.split()) > 1:
                new_file.write(line)

יצחק אולשן



In [100]:
with open('C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/Pipeline Python Scripts/israel_court_judges.txt','r') as file:
    lines = file.readlines()
    
    with open('C:/Users/Itai/HebrewCourtVerdictsAnalyzer/ML/Pipeline Python Scripts/israel_court_judges_fixed.txt','w') as nfile:

        for line in lines:

            splitted = line.split()
            
            if len(splitted) >= 3:
                
                corrected = f'{splitted[2]} {splitted[0]}-{splitted[1]}'

            else:
                corrected = f'{splitted[0]} {splitted[1]}'
            
            nfile.write(corrected+'\n')
